<a href="https://colab.research.google.com/github/bruno774/doutorado/blob/main/ppgti_exercicioCNN_emnist1_5_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade da Disciplina PPGTI3003 - Aprendizado Profundo - T01 (2025.2)
## Prof. Josenalde Barbosa de Oliveira

Implementar uma arquitetura de Rede Neural Convolucional (CNN) baseada nos experimentos descritos no artigo *Classificação de Caracteres Manuscritos para Correção Automática do Sistema Multiprova (Silva Filho e outros)*.

Foi identificada no artigo qual a melhor arquitetura para o problema de classificação dos dígitos 1 a 5, V e F, A a E como sendo uma CNN identificada como Estrutura 4. O modelo deve ser treinado e testado no conjunto de dados EMNIST, com foco específico nos dígitos de 1 a 5.

A tarefa inclui baixar e preparar o conjunto de dados EMNIST, implementar a CNN em PyTorch, treinar o modelo, avaliar seu desempenho com métricas como acurácia e visualizar os resultados (curvas de perda/acurácia e, potencialmente, previsões).

## Definir Arquitetura CNN


### Detalhes arquiteturais identificados no artigo

As especificações arquiteturais para a 'Estrutura 4' do artigo `silvafilho2022.pdf` definem a seguinte estrutura:
1.  **Camadas Convolucionais (`Conv2d`):** 4 blocos (32 -> 64 -> 64 -> 64 filtros (canais de saída)), tamanho do kernel 3x3 com padding.
2.  **Camadas de Pooling (`MaxPool2d`):
):** MaxPooling 2x2.
3.  **Função de Ativação:** ReLU.
4.  **Camadas Totalmente Conectadas (Fully Connected Layers):** 256 -> 5 classes.
5.  **Camada de Saída:** Número de classes (que é 5, para dígitos 1-5) e função de ativação final (se aplicável, e.g., Softmax).

Camada Flatten

Configurações de treinamento alcançado no experimento:

    * Optimizer: Adam
    * Imagens de entrada: 32x32x3 pixels
    * Early stopping com margem de 0,1%
    * Divisão do dataset: 70% treino, 15% validação, 15% teste

### carregando bibliotecas do projeto

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
from torchvision.datasets import EMNIST
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tqdm import tqdm
import os

### estruturas e defs

In [2]:
class CNNEstrutura4(nn.Module):
    """
    Arquitetura CNN Estrutura 4 conforme descrito no artigo.

    Arquitetura:
    - 4 blocos convolucionais (Conv2d + ReLU + MaxPool2d)
    - Camada Flatten
    - Camada totalmente conectada

    Especificações:
    - 1ª camada: 32 filtros 3x3
    - 2ª-4ª camadas: 64 filtros 3x3
    - Max pooling: 2x2
    - Função de ativação: ReLU
    - Input: 32x32x3 (conforme artigo, mas adaptado para grayscale)
    """

    def __init__(self, num_classes=5):
        super(CNNEstrutura4, self).__init__()

        # Bloco Convolucional 1: 32 filtros 3x3
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Bloco Convolucional 2: 64 filtros 3x3
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Bloco Convolucional 3: 64 filtros 3x3
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Bloco Convolucional 4: 64 filtros 3x3
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Camada totalmente conectada
        # Após 4 max poolings de 2x2, 32x32 -> 2x2
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 2 * 2, num_classes)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


class EMNISTDigitsDataset(Dataset):
    """
    Dataset customizado para filtrar apenas dígitos 1-5 do EMNIST
    """

    def __init__(self, emnist_dataset, target_digits=[1, 2, 3, 4, 5]):
        self.data = []
        self.targets = []

        # Filtrar apenas os dígitos desejados
        for idx in range(len(emnist_dataset)):
            img, label = emnist_dataset[idx]
            if label in target_digits:
                self.data.append(img)
                # Remapear labels para 0-4
                self.targets.append(target_digits.index(label))

        print(f"Dataset filtrado: {len(self.data)} amostras dos dígitos {target_digits}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]


def prepare_data(data_dir='./data', batch_size=64):
    """
    Prepara os dados EMNIST conforme especificações do artigo:
    - 70% treino
    - 15% validação
    - 15% teste
    - Imagens 32x32
    """

    # Transformações: redimensionar para 32x32 e normalizar
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Carregar EMNIST digits
    print("Carregando dataset EMNIST...")
    full_dataset = EMNIST(root=data_dir, split='digits', train=True,
                          download=True, transform=transform)

    # Filtrar apenas dígitos 1-5
    filtered_dataset = EMNISTDigitsDataset(full_dataset, target_digits=[1, 2, 3, 4, 5])

    # Dividir dataset: 70% treino, 15% validação, 15% teste
    total_size = len(filtered_dataset)
    train_size = int(0.70 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(
        filtered_dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    print(f"\nDivisão do dataset:")
    print(f"Treino: {train_size} ({train_size/total_size*100:.1f}%)")
    print(f"Validação: {val_size} ({val_size/total_size*100:.1f}%)")
    print(f"Teste: {test_size} ({test_size/total_size*100:.1f}%)")

    # Criar DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader, test_loader


class EarlyStopping:
    """
    Early stopping conforme descrito no artigo:
    Para quando não há melhora de 0.1% na precisão ou perda
    """

    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = model.state_dict().copy()
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.best_model = model.state_dict().copy()
            self.counter = 0


def train_epoch(model, train_loader, criterion, optimizer, device):
    """Treina o modelo por uma época"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc='Treinamento')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        pbar.set_postfix({
            'loss': running_loss / (pbar.n + 1),
            'acc': 100. * correct / total
        })

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc


def validate(model, val_loader, criterion, device):
    """Valida o modelo"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc='Validação'):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc


def train_model(model, train_loader, val_loader, num_epochs=50, learning_rate=0.001, device='cuda'):
    """
    Treina o modelo usando Adam optimizer conforme artigo
    Implementa early stopping
    """

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    early_stopping = EarlyStopping(patience=5, min_delta=0.001)

    # Listas para armazenar histórico
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    print(f"\nIniciando treinamento no dispositivo: {device}")
    print(f"Optimizer: Adam (lr={learning_rate})")
    print(f"Critério: CrossEntropyLoss")
    print(f"Early Stopping: margem de 0.1%\n")

    for epoch in range(num_epochs):
        print(f"\nÉpoca {epoch+1}/{num_epochs}")
        print("-" * 50)

        # Treinar
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)

        # Validar
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        # Armazenar histórico
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f"\nResumo da Época {epoch+1}:")
        print(f"  Treino    - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
        print(f"  Validação - Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        # Early stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print(f"\nEarly stopping acionado na época {epoch+1}")
            model.load_state_dict(early_stopping.best_model)
            break

    return history


def evaluate_model(model, test_loader, device):
    """
    Avalia o modelo no conjunto de teste
    Retorna métricas detalhadas e matriz de confusão
    """
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Avaliação'):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = outputs.max(1)

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100. * correct / total

    # Matriz de confusão
    cm = confusion_matrix(all_labels, all_predictions)

    # Relatório de classificação
    target_names = ['Dígito 1', 'Dígito 2', 'Dígito 3', 'Dígito 4', 'Dígito 5']
    report = classification_report(all_labels, all_predictions,
                                   target_names=target_names,
                                   digits=4)

    return accuracy, cm, report


def plot_training_history(history, save_path='training_history.png'):
    """
    Plota gráficos de perda e precisão conforme artigo
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    epochs = range(1, len(history['train_loss']) + 1)

    # Gráfico de Perda
    ax1.plot(epochs, history['train_loss'], 'b-', label='Treino', linewidth=2)
    ax1.plot(epochs, history['val_loss'], 'r--', label='Validação', linewidth=2)
    ax1.set_xlabel('Épocas', fontsize=12)
    ax1.set_ylabel('Perda', fontsize=12)
    ax1.set_title('Perda do modelo', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Gráfico de Precisão
    ax2.plot(epochs, history['train_acc'], 'b-', label='Treino', linewidth=2)
    ax2.plot(epochs, history['val_acc'], 'r--', label='Validação', linewidth=2)
    ax2.set_xlabel('Épocas', fontsize=12)
    ax2.set_ylabel('Precisão (%)', fontsize=12)
    ax2.set_title('Precisão do modelo', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"\nGráficos de treinamento salvos em: {save_path}")
    plt.close()


def plot_confusion_matrix(cm, save_path='confusion_matrix.png'):
    """
    Plota matriz de confusão conforme artigo
    """
    plt.figure(figsize=(10, 8))

    # Normalizar para percentuais
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

    # Criar heatmap
    sns.heatmap(cm_normalized, annot=True, fmt='.1f', cmap='Blues',
                xticklabels=['1', '2', '3', '4', '5'],
                yticklabels=['1', '2', '3', '4', '5'],
                cbar_kws={'label': 'Percentual (%)'},
                vmin=0, vmax=100)

    plt.xlabel('Predição', fontsize=12)
    plt.ylabel('Real', fontsize=12)
    plt.title('Matriz de Confusão - Dígitos 1-5', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Matriz de confusão salva em: {save_path}")
    plt.close()


### execução de carga dos dados e treinamento

In [3]:
def main():
    """
    Função principal para executar o pipeline completo
    """

    # Configurações
    BATCH_SIZE = 64
    NUM_EPOCHS = 50
    LEARNING_RATE = 0.001
    DATA_DIR = './data'

    # Verificar GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Dispositivo utilizado: {device}")

    # Preparar dados
    print("\n" + "="*70)
    print("PREPARAÇÃO DOS DADOS")
    print("="*70)
    train_loader, val_loader, test_loader = prepare_data(DATA_DIR, BATCH_SIZE)

    # Criar modelo
    print("\n" + "="*70)
    print("CRIAÇÃO DO MODELO - ESTRUTURA 4")
    print("="*70)
    model = CNNEstrutura4(num_classes=5).to(device)

    # Contar parâmetros
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\nParâmetros totais: {total_params:,}")
    print(f"Parâmetros treináveis: {trainable_params:,}")

    print("\nArquitetura do modelo:")
    print(model)

    # Treinar modelo
    print("\n" + "="*70)
    print("TREINAMENTO")
    print("="*70)
    history = train_model(model, train_loader, val_loader,
                         num_epochs=NUM_EPOCHS,
                         learning_rate=LEARNING_RATE,
                         device=device)

    # Plotar histórico de treinamento
    plot_training_history(history)

    # Avaliar modelo
    print("\n" + "="*70)
    print("AVALIAÇÃO NO CONJUNTO DE TESTE")
    print("="*70)
    accuracy, cm, report = evaluate_model(model, test_loader, device)

    print(f"\n{'='*70}")
    print(f"RESULTADOS FINAIS")
    print(f"{'='*70}")
    print(f"\nPrecisão no teste: {accuracy:.2f}%")
    print(f"\nRelatório de Classificação:")
    print(report)

    # Plotar matriz de confusão
    plot_confusion_matrix(cm)

    # Salvar modelo
    model_path = 'cnn_estrutura4_digits_1-5.pth'
    torch.save({
        'model_state_dict': model.state_dict(),
        'accuracy': accuracy,
        'confusion_matrix': cm,
        'history': history
    }, model_path)
    print(f"\nModelo salvo em: {model_path}")

    # Comparação com resultados do artigo
    print(f"\n{'='*70}")
    print("COMPARAÇÃO COM O ARTIGO")
    print(f"{'='*70}")
    print(f"Resultado do artigo (Estrutura 4, dígitos 0-9): 98,84%")
    print(f"Resultado obtido (dígitos 1-5): {accuracy:.2f}%")

    # Calcular perda final
    final_train_loss = history['train_loss'][-1]
    final_val_loss = history['val_loss'][-1]
    print(f"\nPerda final (treino): {final_train_loss:.4f}")
    print(f"Perda final (validação): {final_val_loss:.4f}")
    print(f"Perda no artigo (Estrutura 4): 0,064")

    print(f"\n{'='*70}")
    print("PIPELINE CONCLUÍDO COM SUCESSO!")
    print(f"{'='*70}\n")


if __name__ == '__main__':
    main()

Dispositivo utilizado: cuda

PREPARAÇÃO DOS DADOS
Carregando dataset EMNIST...


100%|██████████| 562M/562M [00:02<00:00, 205MB/s]


Dataset filtrado: 120000 amostras dos dígitos [1, 2, 3, 4, 5]

Divisão do dataset:
Treino: 84000 (70.0%)
Validação: 18000 (15.0%)
Teste: 18000 (15.0%)

CRIAÇÃO DO MODELO - ESTRUTURA 4

Parâmetros totais: 93,957
Parâmetros treináveis: 93,957

Arquitetura do modelo:
CNNEstrutura4(
  (conv_block1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(

Validação: 100%|██████████| 282/282 [00:00<00:00, 422.24it/s]



Resumo da Época 1:
  Treino    - Loss: 0.0540, Acc: 98.25%
  Validação - Loss: 0.0159, Acc: 99.53%

Época 2/50
--------------------------------------------------


...


Resumo da Época 13:
  Treino    - Loss: 0.0022, Acc: 99.92%
  Validação - Loss: 0.0122, Acc: 99.73%

Época 14/50
--------------------------------------------------


Validação: 100%|██████████| 282/282 [00:00<00:00, 423.90it/s]



Resumo da Época 14:
  Treino    - Loss: 0.0029, Acc: 99.91%
  Validação - Loss: 0.0112, Acc: 99.72%

Early stopping acionado na época 14

Gráficos de treinamento salvos em: training_history.png

AVALIAÇÃO NO CONJUNTO DE TESTE


Avaliação: 100%|██████████| 282/282 [00:00<00:00, 418.74it/s]



RESULTADOS FINAIS

Precisão no teste: 99.76%

Relatório de Classificação:
              precision    recall  f1-score   support

    Dígito 1     0.9978    0.9992    0.9985      3578
    Dígito 2     0.9986    0.9972    0.9979      3612
    Dígito 3     0.9966    0.9943    0.9954      3503
    Dígito 4     0.9992    0.9984    0.9988      3646
    Dígito 5     0.9956    0.9986    0.9971      3661

    accuracy                         0.9976     18000
   macro avg     0.9976    0.9975    0.9975     18000
weighted avg     0.9976    0.9976    0.9976     18000

Matriz de confusão salva em: confusion_matrix.png

Modelo salvo em: cnn_estrutura4_digits_1-5.pth

COMPARAÇÃO COM O ARTIGO
Resultado do artigo (Estrutura 4, dígitos 0-9): 98,84%
Resultado obtido (dígitos 1-5): 99.76%

Perda final (treino): 0.0029
Perda final (validação): 0.0112
Perda no artigo (Estrutura 4): 0,064

PIPELINE CONCLUÍDO COM SUCESSO!



### análise dos resultados

In [10]:
def load_results(model_path='cnn_estrutura4_digits_1-5.pth'):
    """Carrega os resultados salvos do modelo"""
    checkpoint = torch.load(model_path, map_location='cuda', weights_only=False)
    return checkpoint


def analyze_confusion_matrix(cm, class_names=['1', '2', '3', '4', '5']):
    """
    Análise detalhada da matriz de confusão
    """
    print("\n" + "="*70)
    print("ANÁLISE DA MATRIZ DE CONFUSÃO")
    print("="*70)

    # Normalizar para percentuais
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

    # Análise diagonal (acertos)
    print("\n📊 Taxa de Acerto por Classe:")
    print("-" * 50)
    for i, class_name in enumerate(class_names):
        accuracy = cm_normalized[i, i]
        print(f"  Dígito {class_name}: {accuracy:.2f}%")

    # Melhor e pior classe
    diagonal = np.diag(cm_normalized)
    best_class = np.argmax(diagonal)
    worst_class = np.argmin(diagonal)

    print(f"\n✅ Melhor classe: Dígito {class_names[best_class]} ({diagonal[best_class]:.2f}%)")
    print(f"⚠️  Pior classe: Dígito {class_names[worst_class]} ({diagonal[worst_class]:.2f}%)")

    # Análise de confusões (fora da diagonal)
    print("\n🔀 Principais Confusões:")
    print("-" * 50)

    confusions = []
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            if i != j and cm_normalized[i, j] > 0:
                confusions.append((i, j, cm_normalized[i, j]))

    # Ordenar por taxa de confusão
    confusions.sort(key=lambda x: x[2], reverse=True)

    # Mostrar top 5 confusões
    for idx, (i, j, conf_rate) in enumerate(confusions[:5], 1):
        print(f"  {idx}. Dígito {class_names[i]} confundido com {class_names[j]}: {conf_rate:.2f}%")

    if len(confusions) == 0:
        print("  Nenhuma confusão significativa detectada!")

    # Erro máximo
    max_confusion = confusions[0][2] if confusions else 0
    print(f"\n❌ Taxa máxima de erro: {max_confusion:.2f}%")

    return cm_normalized, diagonal, confusions


def compare_with_article(accuracy, loss, cm):
    """
    Compara os resultados obtidos com os do artigo
    """
    print("\n" + "="*70)
    print("COMPARAÇÃO COM OS RESULTADOS DO ARTIGO")
    print("="*70)

    # Dados do artigo - Estrutura 4 (dígitos 0-9)
    article_results = {
        'accuracy': 98.84,
        'loss': 0.064,
        'min_class_accuracy': 98.0,
        'max_error': 0.9,
        'epochs': 24,
        'training_time': 150  # segundos
    }

    print("\n📄 Resultados do Artigo (Estrutura 4, dígitos 0-9):")
    print("-" * 50)
    print(f"  Precisão geral: {article_results['accuracy']:.2f}%")
    print(f"  Perda: {article_results['loss']:.3f}")
    print(f"  Taxa mínima por classe: {article_results['min_class_accuracy']:.2f}%")
    print(f"  Erro máximo entre classes: {article_results['max_error']:.2f}%")
    print(f"  Épocas de treinamento: {article_results['epochs']}")
    print(f"  Tempo de treinamento: ~{article_results['training_time']} segundos")

    # Calcular métricas do modelo atual
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    diagonal = np.diag(cm_normalized)
    min_class_acc = diagonal.min()

    # Confusões
    max_error = 0
    for i in range(len(cm_normalized)):
        for j in range(len(cm_normalized)):
            if i != j:
                max_error = max(max_error, cm_normalized[i, j])

    print("\n🔬 Resultados Obtidos (dígitos 1-5):")
    print("-" * 50)
    print(f"  Precisão geral: {accuracy:.2f}%")
    print(f"  Perda: {loss:.3f}")
    print(f"  Taxa mínima por classe: {min_class_acc:.2f}%")
    print(f"  Erro máximo entre classes: {max_error:.2f}%")

    # Análise comparativa
    print("\n📊 Análise Comparativa:")
    print("-" * 50)

    diff_accuracy = accuracy - article_results['accuracy']
    diff_loss = loss - article_results['loss']
    diff_min_acc = min_class_acc - article_results['min_class_accuracy']
    diff_max_error = max_error - article_results['max_error']

    print(f"  Diferença na precisão: {diff_accuracy:+.2f}% ", end="")
    print("✅" if diff_accuracy >= -1 else "⚠️")

    print(f"  Diferença na perda: {diff_loss:+.3f} ", end="")
    print("✅" if diff_loss <= 0.01 else "⚠️")

    print(f"  Diferença taxa mín. classe: {diff_min_acc:+.2f}% ", end="")
    print("✅" if diff_min_acc >= -2 else "⚠️")

    print(f"  Diferença erro máximo: {diff_max_error:+.2f}% ", end="")
    print("✅" if diff_max_error <= 1 else "⚠️")

    # Conclusão
    print("\n💡 Conclusão:")
    print("-" * 50)

    if accuracy >= 98.0 and max_error <= 2.0:
        print("  ✅ Resultados EXCELENTES! Comparáveis ao artigo.")
    elif accuracy >= 97.0 and max_error <= 3.0:
        print("  ✅ Resultados MUITO BONS! Próximos ao artigo.")
    elif accuracy >= 95.0:
        print("  ⚠️  Resultados BONS, mas podem ser melhorados.")
    else:
        print("  ❌ Resultados abaixo do esperado. Revisar configurações.")

    return {
        'article': article_results,
        'obtained': {
            'accuracy': accuracy,
            'loss': loss,
            'min_class_accuracy': min_class_acc,
            'max_error': max_error
        },
        'differences': {
            'accuracy': diff_accuracy,
            'loss': diff_loss,
            'min_class_accuracy': diff_min_acc,
            'max_error': diff_max_error
        }
    }


def analyze_training_history(history):
    """
    Análise detalhada do histórico de treinamento
    """
    print("\n" + "="*70)
    print("ANÁLISE DO HISTÓRICO DE TREINAMENTO")
    print("="*70)

    train_loss = history['train_loss']
    val_loss = history['val_loss']
    train_acc = history['train_acc']
    val_acc = history['val_acc']

    num_epochs = len(train_loss)

    print(f"\n📈 Estatísticas Gerais:")
    print("-" * 50)
    print(f"  Número de épocas: {num_epochs}")
    print(f"  Perda inicial (treino): {train_loss[0]:.4f}")
    print(f"  Perda final (treino): {train_loss[-1]:.4f}")
    print(f"  Perda inicial (validação): {val_loss[0]:.4f}")
    print(f"  Perda final (validação): {val_loss[-1]:.4f}")
    print(f"  Melhoria na perda: {(train_loss[0] - train_loss[-1])/train_loss[0]*100:.1f}%")

    print(f"\n  Precisão inicial (treino): {train_acc[0]:.2f}%")
    print(f"  Precisão final (treino): {train_acc[-1]:.2f}%")
    print(f"  Precisão inicial (validação): {val_acc[0]:.2f}%")
    print(f"  Precisão final (validação): {val_acc[-1]:.2f}%")
    print(f"  Melhoria na precisão: {train_acc[-1] - train_acc[0]:.2f}%")

    # Detectar overfitting
    print(f"\n🔍 Análise de Overfitting:")
    print("-" * 50)

    gap_loss = train_loss[-1] - val_loss[-1]
    gap_acc = train_acc[-1] - val_acc[-1]

    print(f"  Gap perda (treino - validação): {gap_loss:.4f}")
    print(f"  Gap precisão (treino - validação): {gap_acc:.2f}%")

    if abs(gap_loss) < 0.05 and abs(gap_acc) < 2:
        print("  ✅ Sem sinais de overfitting significativo")
    elif abs(gap_loss) < 0.1 and abs(gap_acc) < 5:
        print("  ⚠️  Leve overfitting detectado")
    else:
        print("  ❌ Overfitting significativo detectado")

    # Convergência
    print(f"\n🎯 Análise de Convergência:")
    print("-" * 50)

    if num_epochs >= 10:
        last_5_loss_change = abs(val_loss[-1] - val_loss[-5])
        last_5_acc_change = abs(val_acc[-1] - val_acc[-5])

        print(f"  Variação perda (últimas 5 épocas): {last_5_loss_change:.4f}")
        print(f"  Variação precisão (últimas 5 épocas): {last_5_acc_change:.2f}%")

        if last_5_loss_change < 0.01 and last_5_acc_change < 0.5:
            print("  ✅ Modelo convergiu adequadamente")
        else:
            print("  ⚠️  Modelo ainda estava melhorando")

    # Melhor época
    best_epoch = np.argmin(val_loss)
    print(f"\n⭐ Melhor Época: {best_epoch + 1}")
    print("-" * 50)
    print(f"  Perda validação: {val_loss[best_epoch]:.4f}")
    print(f"  Precisão validação: {val_acc[best_epoch]:.2f}%")


def plot_detailed_analysis(history, cm, save_path='detailed_analysis.png'):
    """
    Cria visualização completa da análise
    """
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

    # 1. Perda ao longo das épocas
    ax1 = fig.add_subplot(gs[0, 0])
    epochs = range(1, len(history['train_loss']) + 1)
    ax1.plot(epochs, history['train_loss'], 'b-', label='Treino', linewidth=2)
    ax1.plot(epochs, history['val_loss'], 'r--', label='Validação', linewidth=2)
    ax1.set_xlabel('Épocas')
    ax1.set_ylabel('Perda')
    ax1.set_title('Evolução da Perda', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Precisão ao longo das épocas
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(epochs, history['train_acc'], 'b-', label='Treino', linewidth=2)
    ax2.plot(epochs, history['val_acc'], 'r--', label='Validação', linewidth=2)
    ax2.set_xlabel('Épocas')
    ax2.set_ylabel('Precisão (%)')
    ax2.set_title('Evolução da Precisão', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # 3. Gap entre treino e validação
    ax3 = fig.add_subplot(gs[0, 2])
    gap_loss = [t - v for t, v in zip(history['train_loss'], history['val_loss'])]
    ax3.plot(epochs, gap_loss, 'g-', linewidth=2)
    ax3.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    ax3.set_xlabel('Épocas')
    ax3.set_ylabel('Gap (Treino - Validação)')
    ax3.set_title('Análise de Overfitting (Perda)', fontweight='bold')
    ax3.grid(True, alpha=0.3)

    # 4. Matriz de confusão normalizada
    ax4 = fig.add_subplot(gs[1, :2])
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    sns.heatmap(cm_normalized, annot=True, fmt='.1f', cmap='Blues',
                xticklabels=['1', '2', '3', '4', '5'],
                yticklabels=['1', '2', '3', '4', '5'],
                cbar_kws={'label': 'Percentual (%)'},
                ax=ax4, vmin=0, vmax=100)
    ax4.set_xlabel('Predição')
    ax4.set_ylabel('Real')
    ax4.set_title('Matriz de Confusão Normalizada', fontweight='bold')

    # 5. Precisão por classe
    ax5 = fig.add_subplot(gs[1, 2])
    diagonal = np.diag(cm_normalized)
    classes = ['1', '2', '3', '4', '5']
    colors = ['green' if x >= 98 else 'orange' if x >= 95 else 'red' for x in diagonal]
    bars = ax5.bar(classes, diagonal, color=colors, alpha=0.7)
    ax5.axhline(y=98, color='r', linestyle='--', alpha=0.5, label='Meta (98%)')
    ax5.set_xlabel('Dígito')
    ax5.set_ylabel('Precisão (%)')
    ax5.set_title('Precisão por Classe', fontweight='bold')
    ax5.legend()
    ax5.grid(True, alpha=0.3, axis='y')

    # Adicionar valores nas barras
    for bar in bars:
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom')

    # 6. Histórico de melhoria
    ax6 = fig.add_subplot(gs[2, 0])
    improvements_loss = []
    for i in range(1, len(history['val_loss'])):
        improvement = history['val_loss'][i-1] - history['val_loss'][i]
        improvements_loss.append(improvement)

    ax6.bar(range(2, len(history['val_loss']) + 1), improvements_loss,
            color=['green' if x > 0 else 'red' for x in improvements_loss], alpha=0.7)
    ax6.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax6.set_xlabel('Época')
    ax6.set_ylabel('Melhoria na Perda')
    ax6.set_title('Melhoria Época a Época', fontweight='bold')
    ax6.grid(True, alpha=0.3, axis='y')

    # 7. Learning curve
    ax7 = fig.add_subplot(gs[2, 1:])
    ax7.plot(epochs, history['train_loss'], 'b-', label='Perda Treino', linewidth=2)
    ax7.plot(epochs, history['val_loss'], 'r--', label='Perda Validação', linewidth=2)
    ax7_twin = ax7.twinx()
    ax7_twin.plot(epochs, history['train_acc'], 'g-', label='Acc Treino',
                  linewidth=2, alpha=0.5)
    ax7_twin.plot(epochs, history['val_acc'], 'm--', label='Acc Validação',
                  linewidth=2, alpha=0.5)

    ax7.set_xlabel('Épocas')
    ax7.set_ylabel('Perda', color='b')
    ax7_twin.set_ylabel('Precisão (%)', color='g')
    ax7.set_title('Learning Curve Completa', fontweight='bold')
    ax7.legend(loc='upper left')
    ax7_twin.legend(loc='upper right')
    ax7.grid(True, alpha=0.3)

    plt.suptitle('Análise Detalhada do Treinamento - Estrutura 4',
                fontsize=16, fontweight='bold', y=0.995)

    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"\n📊 Visualização detalhada salva em: {save_path}")
    plt.close()


def generate_report(checkpoint, save_path='analysis_report.txt'):
    """
    Gera relatório completo em texto
    """
    with open(save_path, 'w', encoding='utf-8') as f:
        f.write("="*70 + "\n")
        f.write("RELATÓRIO DE ANÁLISE - CNN ESTRUTURA 4\n")
        f.write("Classificação de Dígitos 1-5 (EMNIST)\n")
        f.write("Baseado em: Silva Filho et al. (2022)\n")
        f.write("="*70 + "\n\n")

        # Métricas principais
        f.write("MÉTRICAS PRINCIPAIS\n")
        f.write("-"*70 + "\n")
        f.write(f"Precisão no teste: {checkpoint['accuracy']:.2f}%\n")

        cm = checkpoint['confusion_matrix']
        history = checkpoint['history']

        f.write(f"Perda final (treino): {history['train_loss'][-1]:.4f}\n")
        f.write(f"Perda final (validação): {history['val_loss'][-1]:.4f}\n")
        f.write(f"Número de épocas: {len(history['train_loss'])}\n\n")

        # Análise por classe
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
        diagonal = np.diag(cm_normalized)

        f.write("PRECISÃO POR CLASSE\n")
        f.write("-"*70 + "\n")
        for i, acc in enumerate(diagonal, 1):
            f.write(f"Dígito {i}: {acc:.2f}%\n")

        f.write(f"\nMelhor classe: Dígito {np.argmax(diagonal)+1} ({diagonal.max():.2f}%)\n")
        f.write(f"Pior classe: Dígito {np.argmin(diagonal)+1} ({diagonal.min():.2f}%)\n\n")

        # Comparação com artigo
        f.write("COMPARAÇÃO COM O ARTIGO\n")
        f.write("-"*70 + "\n")
        f.write("Artigo (Estrutura 4, dígitos 0-9): 98,84%\n")
        f.write(f"Obtido (dígitos 1-5): {checkpoint['accuracy']:.2f}%\n")
        f.write(f"Diferença: {checkpoint['accuracy'] - 98.84:+.2f}%\n\n")

        f.write("Artigo (perda): 0,064\n")
        f.write(f"Obtido (perda validação): {history['val_loss'][-1]:.4f}\n")
        f.write(f"Diferença: {history['val_loss'][-1] - 0.064:+.4f}\n\n")

        # Conclusão
        f.write("CONCLUSÃO\n")
        f.write("-"*70 + "\n")
        if checkpoint['accuracy'] >= 98.0:
            f.write("✅ Resultados EXCELENTES! Comparáveis ao artigo original.\n")
            f.write("A arquitetura Estrutura 4 demonstrou alta eficácia na\n")
            f.write("classificação dos dígitos 1-5 do EMNIST.\n")
        elif checkpoint['accuracy'] >= 97.0:
            f.write("✅ Resultados MUITO BONS! Próximos ao artigo original.\n")
        else:
            f.write("⚠️ Resultados podem ser melhorados.\n")

    print(f"\n📝 Relatório completo salvo em: {save_path}")

In [11]:
def main():
    """
    Função principal para análise completa
    """
    print("\n" + "="*70)
    print("ANÁLISE DETALHADA DOS RESULTADOS")
    print("CNN Estrutura 4 - Dígitos 1-5 (EMNIST)")
    print("="*70)

    try:
        # Carregar resultados
        print("\n📂 Carregando resultados...")
        checkpoint = load_results()

        accuracy = checkpoint['accuracy']
        cm = checkpoint['confusion_matrix']
        history = checkpoint['history']

        # Calcular perda final
        final_loss = history['val_loss'][-1]

        # Análise da matriz de confusão
        cm_norm, diagonal, confusions = analyze_confusion_matrix(cm)

        # Comparação com artigo
        comparison = compare_with_article(accuracy, final_loss, cm)

        # Análise do histórico
        analyze_training_history(history)

        # Gerar visualizações
        print("\n📊 Gerando visualizações...")
        plot_detailed_analysis(history, cm)

        # Gerar relatório
        print("\n📝 Gerando relatório...")
        generate_report(checkpoint)

        print("\n" + "="*70)
        print("ANÁLISE CONCLUÍDA COM SUCESSO!")
        print("="*70)
        print("\nArquivos gerados:")
        print("  - detailed_analysis.png (visualizações)")
        print("  - analysis_report.txt (relatório)")

    except FileNotFoundError:
        print("\n❌ Erro: Arquivo do modelo não encontrado!")
        print("Execute primeiro: python cnn_emnist_digits.py")
    except Exception as e:
        print(f"\n❌ Erro durante a análise: {str(e)}")


if __name__ == '__main__':
    main()



ANÁLISE DETALHADA DOS RESULTADOS
CNN Estrutura 4 - Dígitos 1-5 (EMNIST)

📂 Carregando resultados...

ANÁLISE DA MATRIZ DE CONFUSÃO

📊 Taxa de Acerto por Classe:
--------------------------------------------------
  Dígito 1: 99.92%
  Dígito 2: 99.72%
  Dígito 3: 99.43%
  Dígito 4: 99.84%
  Dígito 5: 99.86%

✅ Melhor classe: Dígito 1 (99.92%)
⚠️  Pior classe: Dígito 3 (99.43%)

🔀 Principais Confusões:
--------------------------------------------------
  1. Dígito 3 confundido com 5: 0.37%
  2. Dígito 2 confundido com 3: 0.17%
  3. Dígito 5 confundido com 3: 0.14%
  4. Dígito 3 confundido com 1: 0.09%
  5. Dígito 2 confundido com 1: 0.08%

❌ Taxa máxima de erro: 0.37%

COMPARAÇÃO COM OS RESULTADOS DO ARTIGO

📄 Resultados do Artigo (Estrutura 4, dígitos 0-9):
--------------------------------------------------
  Precisão geral: 98.84%
  Perda: 0.064
  Taxa mínima por classe: 98.00%
  Erro máximo entre classes: 0.90%
  Épocas de treinamento: 24
  Tempo de treinamento: ~150 segundos

🔬 Resul

AttributeError: module 'torch.nn' has no attribute 'torchsummary'

### Sumário da Arquitetura do Modelo

Esta seção apresenta a arquitetura da CNN 'Estrutura 4' implementada, conforme especificado no artigo, e resume a contagem de parâmetros do modelo. Isso é útil para verificar se a estrutura do modelo está correta e para entender a complexidade do modelo.

In [13]:
import torch
import torch.nn as nn

# Instantiate the model (assuming CNNEstrutura4 class is already defined and executed)
model_summary = CNNEstrutura4(num_classes=5)

# Count parameters
total_params = sum(p.numel() for p in model_summary.parameters())
trainable_params = sum(p.numel() for p in model_summary.parameters() if p.requires_grad)

print("\n" + "="*70)
print("RESUMO DA ARQUITETURA DO MODELO")
print("="*70)

print(f"\nParâmetros totais: {total_params:,}")
print(f"Parâmetros treináveis: {trainable_params:,}")

print("\nArquitetura do modelo:")
print(model_summary)


RESUMO DA ARQUITETURA DO MODELO

Parâmetros totais: 93,957
Parâmetros treináveis: 93,957

Arquitetura do modelo:
CNNEstrutura4(
  (conv_block1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_di